# 👋🌍 Hello, world: Submit an optimization job to Microsoft QIO

This notebook will walk through how to use Azure Quantum to submit optimization problems to [Microsoft QIO](https://docs.microsoft.com/azure/quantum/optimization-overview-introduction).

## Submit a simple job to Microsoft QIO using Azure Quantum

Let's begin. All code in this example will be written in Python. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace
The first step is connecting to your Azure Quantum workspace. To do so, simply create a `Workspace` object as seen in the following code cell. The parameters corresponding to your workspace should have been automatically entered for you.

In [ ]:
from azure.quantum import Workspace

workspace = Workspace (
    subscription_id = "",
    resource_group = "",
    name = "",
    location = ""
)

Let's use the `Workspace` object to view the available _targets_.

In [ ]:
targets = workspace.get_targets()
print("This workspace's targets:")
for target in targets:
    print("-", target.name)

The resulting list should contain a list of target names, along with some details for each one. A _target_ is a remote solver that can perform optimization or quantum computation. In this example, we will focus on optimization.

### ❕ Do you see `microsoft.simulatedannealing-parameterfree.cpu` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Microsoft QIO to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Microsoft QIO provider.

### Microsoft QIO: The optimization provider
Azure Quantum connects with first- and third-party optimizers to deliver optimization solutions. These offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Microsoft QIO's targets.

**Microsoft QIO target ID anatomy**: `microsoft.{algorithm name}.{hardware type}`. Algorithm names ending in `-parameterfree` use automatic parameter selection. 

Optimization Algorithm | Target IDs | Python Class
| --- | ---| ---| --- | -- |
[Simulated annealing](https://docs.microsoft.com/azure/quantum/optimization-simulated-annealing) | `microsoft.simulatedannealing.cpu`,<br/>`microsoft.simulatedannealing-parameterfree.cpu`,<br/>`microsoft.simulatedannealing.fpga` |  `azure.quantum.optimization.SimulatedAnnealing` |
[Parallel tempering](https://docs.microsoft.com/azure/quantum/optimization-parallel-tempering) | `microsoft.paralleltempering.cpu`,<br/>`microsoft.paralleltempering-parameterfree.cpu`,<br/>`microsoft.paralleltempering.fpga`  | `azure.quantum.optimization.ParallelTempering`
[Tabu search](https://docs.microsoft.com/en-us/azure/quantum/optimization-tabu-search) | `microsoft.tabu.cpu`,<br/>`microsoft.tabu-parameterfree.cpu`  | `azure.quantum.optimization.Tabu`
[Quantum Monte Carlo](https://docs.microsoft.com/azure/quantum/optimization-quantum-monte-carlo) | `microsoft.qmc.cpu`  | `azure.quantum.optimization.QuantumMonteCarlo`
[Population annealing](https://docs.microsoft.com/azure/quantum/optimization-population-annealing) | `microsoft.populationannealing.cpu`  | `azure.quantum.optimization.PopulationAnnealing`
[Substochastic Monte Carlo](https://docs.microsoft.com/azure/quantum/optimization-substochastic-monte-carlo) | `microsoft.substochasticmontecarlo.cpu`  | `azure.quantum.optimization.SubstochasticMonteCarlo`

For this example, we will use `microsoft.simulatedannealing-parameterfree.cpu`. To learn more about Microsoft QIO's targets, check out our [documentation](https://docs.microsoft.com/azure/quantum/provider-microsoft-qio).

### 2. Define an optimization job
We will demonstrate how to use Azure Quantum for optimization by submitting a very simple cost function. Let's define one that contains just two terms.

In [ ]:
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term

terms = [Term(c=1, indices=[1, 0]), Term(c=2, indices=[0, 1])]
problem = Problem(name="Optimization problem", problem_type=ProblemType.ising, terms=terms)

print("Name:", problem.name)
print("Number of terms:", len(problem.terms))
print("Problem type:", problem.problem_type)

You should see the problem details in the output. Note that this problem uses `ProblemType.ising`, which means it uses an [Ising model](https://docs.microsoft.com/azure/quantum/optimization-concepts-ising-model-for-optimization). [Binary optimization](https://docs.microsoft.com/azure/quantum/optimization-binary-optimization) formulations (such as PUBO) are also supported. Read more about problem types [here](https://docs.microsoft.com/azure/quantum/optimization-problem-type).  

### 3. Submit the job
Now that we've defined a problem, we can submit it to Azure Quantum. Note that this will consume a few seconds of computation time, but all workspaces get one free hour of computation on Microsoft QIO (and the problem should only take a few seconds).

In [ ]:
from azure.quantum.optimization import SimulatedAnnealing
import time

# Instantiate a solver to solve the problem. 
solver = SimulatedAnnealing(workspace, timeout=100)

# Optimize the problem.
print('Submitting problem...')
start = time.time()
result = solver.optimize(problem)
time_elapsed = time.time() - start
solutions = result['solutions']

# Print the solution(s).
print(f'\nObtained result in {time_elapsed} seconds.')
print(f'Computed {len(solutions)} solution(s).')
for i in range(len(solutions)):
    print(f'Solution {i}:')
    print('\tCost:', solutions[i]['cost'])
    print('\tConfiguration:', solutions[i]['configuration'])

**See a solutiuon listed above? Congratulations, you've submitted an optimization job with Azure Quantum! 👏**

### 4. Next steps
Next, you can try submitting an optimization job to other solvers that Microsoft QIO offers. Or try submitting to another provider entirely by navigating back to the sample gallery. The same "hello world" sample can be run with different optimization providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.